# Creating municipality RIDA layers

Assuming that the data we received from Septima by WeTransfer (2 folders with fgb files, plus one separate fgb file) is in the folder `data/RIDA` (from Dropbox [here](https://www.dropbox.com/scl/fo/38gkcvyfex0atdbqe3hfn/h?rlkey=z9xwq2mh6ga03c301b7f66wqx&dl=0))

This script imports each of the fgb layers we need and cuts them into muncipality-sized bites.

In [109]:
# import packages
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
# read in kommuner data and make subfolders

muni = gpd.read_file("data/raw/municipalities/muni_boundary.gpkg")
muni.head()

,id.namespace,id.lokalId,status,geometristatus,virkningFra,virkningTil,virkningsaktoer,forretningshaendelse,registreringFra,registreringTil,...,landekode,skala,udtraeksdato,kommunekode,LAU1vaerdi,udenforKommuneinddeling,regionskode,regionsLokalId,region,geometry
0,http://data.gov.dk/dagi,389148,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 09:46:41,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0370,370,false,1085,389100,NaN,"MULTIPOLYGON (((672902.450 6143708.870, 672880..."
1,http://data.gov.dk/dagi,389144,vedtaget,endelig,2022-01-01,NaT,Geodatastyrelsen,ændretBekendtgørelse,2021-12-14 09:46:41,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0330,330,false,1085,389100,NaN,"MULTIPOLYGON (((654675.250 6154223.210, 654636..."
2,http://data.gov.dk/dagi,389178,vedtaget,endelig,2021-02-19,NaT,Økonomi- og Indenrigsministeriet,fejlrettelseØvrige,2021-02-19 09:03:56,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0766,766,false,1082,389101,NaN,"MULTIPOLYGON (((563379.450 6175110.360, 563346..."
3,http://data.gov.dk/dagi,389166,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 10:23:11,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0510,510,false,1083,389102,NaN,"MULTIPOLYGON (((516449.610 6137370.130, 516423..."
4,http://data.gov.dk/dagi,389168,vedtaget,endelig,2022-01-01,NaT,Økonomi- og Indenrigsministeriet,ændretBekendtgørelse,2021-12-14 10:23:11,NaT,...,DK,1:10.000,2022-02-04 14:50:06,0550,550,false,1083,389102,NaN,"MULTIPOLYGON (((482562.400 6122027.620, 482507..."


<img src="struct.png" width=60%>


In [3]:
for _, row in muni.iterrows():
    os.makedirs(f"data/raw/RIDA/{row.kommunekode}", exist_ok=True)

In [9]:
rida_dict = {
    # "facilit_indkoeb.fgb": "indkoeb.gpkg",
    # "facilit_overnatning.fgb": "overnatning.gpkg",
    # "facilit_rasteplads.fgb": "rasteplads.gpkg",
    # 'facilit_service.fgb': "infoservice.gpkg",
    # 'land_anvendelse.fgb': "anvendelse.gpkg", # new! formerly 5 layers (byomrade, dyrket areal, naturareal, skovinddeling, teknisk areal)
    # 'land_beskyttnatur_flade.fgb': "beskyttet_natur.gpkg",
    'land_beskyttnatur_linje.fgb': "beskyttet_vandloeb.gpkg",
    'land_besoegs.fgb': "besoeg.gpkg",
    "land_fortid.fgb": "fortidsminder.gpkg",
    'land_frednatpark.fgb': "frednatpark.gpkg", # new!
    'land_landemaerke.fgb': "landemaerker.gpkg",
    'land_udflugt.fgb': "udflugt.gpkg",
    'land_vaerdifuld.fgb': "vaerdifuld.gpkg", # new!
    'facilit_overnatning.fgb': "overnatning.gpkg",    
}


In [10]:
%%time
for k, v in rida_dict.items():
    print(f"reading {k}")
    gdf = gpd.read_file(f"data/RIDA/{k}")
    assert gdf.crs == muni.crs
    for _, row in muni.iterrows():
        gdf_curr = gdf.copy()
        gdf_curr = gdf_curr.clip(row.geometry)
        gdf_curr = gdf_curr.reset_index(drop=True)
        if not gdf_curr.empty:
            gdf_curr.to_file(f"data/raw/RIDA/{row.kommunekode}/{v}")    

reading land_beskyttnatur_linje.fgb
reading land_besoegs.fgb
reading land_fortid.fgb
reading land_frednatpark.fgb
reading land_landemaerke.fgb
reading land_udflugt.fgb
reading land_vaerdifuld.fgb
reading facilit_overnatning.fgb
CPU times: user 7min 30s, sys: 15.2 s, total: 7min 45s
Wall time: 7min 53s


In [ ]:
# for each file type, remove not needed columns to reduce file size
# and (potentially) aggregate types

In [58]:
kode = "0101"

In [ ]:
def merge_gdfs(gdf_list):
    # make sure all gdfs are the same crs
    assert len(set([gdf.crs for gdf in gdf_list])) == 1
    # make sure we have the same geometries everywhere
    assert len(set([t for gdf in gdf_list for t in gdf.type])) == 1
    # concatenate with pandas
    gdf_main = pd.concat([gdf[["gruppe", "type", "geometry"]].copy() for gdf in gdf_list])
    gdf_main = gdf_main.reset_index(drop=True)
    return gdf_main

In [105]:
for kode in list(muni.kommunekode):
    print(f"start kode {kode}")

    anve = gpd.read_file(f"data/raw/RIDA/{kode}/anvendelse.gpkg")
    anve = anve[["gruppe", "type", "geometry"]]
    anve = anve.explode(index_parts=False)
    anve = anve[anve.geometry.type == "Polygon"].copy().reset_index(drop=True)

    besk = gpd.read_file(f"data/raw/RIDA/{kode}/beskyttet_natur.gpkg")
    besk = besk[["type", "geometry"]]
    besk["gruppe"] = "beskyttet_natur"
    besk = besk.explode(index_parts=False)
    besk = besk[besk.geometry.type == "Polygon"].copy().reset_index(drop=True)

    beso = gpd.read_file(f"data/raw/RIDA/{kode}/besoeg.gpkg")
    beso = beso[["gruppe", "type", "geometry"]]
    beso = beso.explode(index_parts=False)
    beso = beso[beso.geometry.type == "Point"].copy().reset_index(drop=True)

    fort = gpd.read_file(f"data/raw/RIDA/{kode}/fortidsminder.gpkg")
    fort = fort[["gruppe", "type", "geometry"]]
    fort = fort.explode(index_parts=False)
    fort = fort[fort.geometry.type == "Point"].copy().reset_index(drop=True)

    fred = gpd.read_file(f"data/raw/RIDA/{kode}/frednatpark.gpkg")
    fred = fred[["type", "geometry"]]
    fred["gruppe"] = "frednatpark"
    fred = fred.explode(index_parts=False)
    fred = fred[fred.geometry.type == "Polygon"].copy().reset_index(drop=True)

    indk = gpd.read_file(f"data/raw/RIDA/{kode}/indkoeb.gpkg")
    indk = indk[["type", "geometry"]]
    indk["gruppe"] = "indkoeb"
    indk = indk.explode(index_parts=False)
    indk = indk[indk.geometry.type == "Point"].copy().reset_index(drop=True)

    info = gpd.read_file(f"data/raw/RIDA/{kode}/infoservice.gpkg")
    info = info[["gruppe", "type", "geometry"]]
    info = info.explode(index_parts=False)
    info = info[info.geometry.type == "Point"].copy().reset_index(drop=True)

    land = gpd.read_file(f"data/raw/RIDA/{kode}/landemaerker.gpkg")
    land = land[["type", "geometry"]]
    land["gruppe"] = "landemaerker"
    land = land.explode(index_parts=False)
    land = land[land.geometry.type == "Point"].copy().reset_index(drop=True)

    over = gpd.read_file(f"data/raw/RIDA/{kode}/overnatning.gpkg")
    over = over[["type", "geometry"]]
    over["gruppe"] = "overnatning"
    over = over.explode(index_parts=False)
    over = over[over.geometry.type == "Point"].copy().reset_index(drop=True)

    rast = gpd.read_file(f"data/raw/RIDA/{kode}/rasteplads.gpkg")
    rast = rast[["gruppe", "type", "geometry"]]
    rast = rast.explode(index_parts=False)
    rast = rast[rast.geometry.type == "Point"].copy().reset_index(drop=True)

    udfl = gpd.read_file(f"data/raw/RIDA/{kode}/udflugt.gpkg")
    udfl = udfl[["gruppe", "type", "geometry"]]
    udfl = udfl.explode(index_parts=False)
    udfl = udfl[udfl.geometry.type == "Point"].copy().reset_index(drop=True)

    vaer = gpd.read_file(f"data/raw/RIDA/{kode}/vaerdifuld.gpkg")
    vaer = vaer[["type", "geometry"]]
    vaer["gruppe"] = "vaerdifuld"
    vaer = vaer.explode(index_parts=False)
    vaer = vaer[vaer.geometry.type == "Polygon"].copy().reset_index(drop=True)

    print("\t data read in")

    nature = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Naturtyper") & -anve["type"].isin(["Råstofområde"])].copy(),
            anve[anve["gruppe"]=="Skovinddeling"].copy(),
            besk.copy(),
            vaer[vaer["type"]=="Naturbeskyttelsesområde"].copy(),
            fred.copy()
        ]
    )

    if len(nature) > 0:
        nature.to_file(f"data/raw/RIDA/{kode}/nature.gpkg", index = False)
    
        
    culture = merge_gdfs(
        [
            anve[anve["type"].isin(['Bykerne'])].copy(),
            vaer[vaer["type"].isin(
                [
                    'Kulturhistorisk bevaringsværdi', 
                    'Værdifuldt kulturmiljø', 
                    'Bevaringsværdig landskab'
                ]
            )].copy()
        ]
    )

    if len(culture) > 0:
        culture.to_file(
            f"data/raw/RIDA/{kode}/culture.gpkg", 
            index = False)
        
    sommerhus = anve[
        anve["type"].isin(
            ['Sommerhusområde', 'Sommerhusområde skov']
            )
        ].copy().reset_index(drop=True)
    if len(sommerhus) > 0:
        sommerhus.to_file(
            f"data/raw/RIDA/{kode}/sommerhus.gpkg", 
            index = False)

    agri = anve[
        anve["gruppe"]=="Dyrkede arealer"].copy().reset_index(
            drop=True)

    if len(agri) > 0:
        agri.to_file(
            f"data/raw/RIDA/{kode}/agriculture.gpkg", 
            index = False)
        
    bad = merge_gdfs(
        [
            anve[(anve["gruppe"]=="Teknisk areal") & (-anve["type"].isin(["Sportsanlæg"]))].copy(),
            anve[(anve["gruppe"]=="Bymæssig anvendelse") & (anve["type"].isin(["Høj bebyggelse", "Erhverv"]))].copy()
        ]
    )

    if len(bad) > 0:
        bad.to_file(
            f"data/raw/RIDA/{kode}/bad.gpkg", 
            index = False)
        

    pois = merge_gdfs(
        [beso, fort, udfl, land]
    )

    pois.plot()

    if len(pois) > 0:
        pois.to_file(
            f"data/raw/RIDA/{kode}/pois.gpkg", 
            index = False)
        
    faci = merge_gdfs(
        [
            info[-info["type"].isin(["Turistkontor"])].copy(),
            rast,
            indk
        ]
    )

    if len(faci) > 0:
        faci.to_file(
            f"data/raw/RIDA/{kode}/facilities.gpkg", 
            index = False)
        
    service = merge_gdfs(
        [indk, over, info[info["type"].isin(["Turistkontor"])]]
    )

    if len(service) > 0:
        service.to_file(
            f"data/raw/RIDA/{kode}/service.gpkg", 
            index = False)
        
    print(f"\t {kode} done")

### PICK UP HERE:

* ~~add all (needed) fgb files to rida_dict~~
* ~~run to create separate layers, PUSH TO MAIN SEPARATELY!~~
* ~~add a code to save kommune polygons separately~~~
* ~~convert 00_TEMP_study_area.ipynb to .py file~~
* adjust 00 file to importing data by kommunekode (config file)
* move 03a to archive
* edit 03b, convert to 03a, adding new layers!
* check if script works
* merge (?) `niras_wfs_exports` and `folkersma_digital`? 